# RAG 검색 시스템 구축 - AI Agent & RAG 주제

## Milestone 1: 문서 수집 및 로드
- 주제: AI Agent & RAG 관련 기술 블로그
- 문서 수: 5개 (LangChain 공식 문서 및 블로그)

In [3]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API KEY 정보로드
load_dotenv()

True

In [4]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


## 1. 문서 로드 - AI Agent & RAG 기술 블로그

In [5]:
# AI Agent & RAG 관련 기술 블로그 URL
urls = [
    "https://python.langchain.com/docs/tutorials/rag/",
    "https://python.langchain.com/docs/concepts/vectorstores/",
    "https://python.langchain.com/docs/concepts/text_splitters/",
    "https://python.langchain.com/docs/concepts/embedding_models/",
    "https://blog.langchain.dev/deconstructing-rag/"
]

# Document Loader 설정
loader = WebBaseLoader(
    web_paths=urls,
    # bs_kwargs=dict(
    #     parse_only=bs4.SoupStrainer(
    #         class_=("post-content", "post-title", "post-header", "article")
    #     )
    # ),
)

# 문서 로드
docs = loader.load()

# 로드 확인
print(f"총 {len(docs)}개의 문서가 로드되었습니다.\n")
for i, doc in enumerate(docs):
    print(f"[문서 {i+1}]")
    print(f"URL: {doc.metadata.get('source', 'Unknown')}")
    print(f"내용 길이: {len(doc.page_content)} characters")
    print(f"내용 미리보기: {doc.page_content[:200]}...")
    print("-" * 80)

총 5개의 문서가 로드되었습니다.

[문서 1]
URL: https://python.langchain.com/docs/tutorials/rag/
내용 길이: 22360 characters
내용 미리보기: Build a RAG agent with LangChain - Docs by LangChainSkip to main contentüöÄ Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangChain + LangGraphSearch....
--------------------------------------------------------------------------------
[문서 2]
URL: https://python.langchain.com/docs/concepts/vectorstores/
내용 길이: 3912 characters
내용 미리보기: LangChain overview - Docs by LangChainSkip to main contentðŸš€ Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangChain + LangGraphSearch...âŒ˜KAsk AIGi...
--------------------------------------------------------------------------------
[문서 3]
URL: https://python.langchain.com/docs/concepts/text_splitters/
내용 길이: 3912 characters
내용 미리보기: LangChain overview - Docs by LangChainSkip to main contentðŸš€ Share how you're building agents for a chance to 

## 2. 청킹 전략 비교 (Milestone 2)

### 2-1. CharacterTextSplitter

In [6]:
# CharacterTextSplitter 적용
char_text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len
)

char_splits = char_text_splitter.split_documents(docs)

print(f"CharacterTextSplitter 결과:")
print(f"- 총 청크 수: {len(char_splits)}")
print(f"- 첫 번째 청크 길이: {len(char_splits[0].page_content)} characters")
print(f"- 첫 번째 청크 미리보기:\n{char_splits[0].page_content[:300]}...")

Created a chunk of size 1250, which is longer than the specified 1000
Created a chunk of size 1106, which is longer than the specified 1000
Created a chunk of size 1817, which is longer than the specified 1000
Created a chunk of size 2111, which is longer than the specified 1000
Created a chunk of size 2111, which is longer than the specified 1000
Created a chunk of size 2111, which is longer than the specified 1000
Created a chunk of size 10640, which is longer than the specified 1000


CharacterTextSplitter 결과:
- 총 청크 수: 42
- 첫 번째 청크 길이: 1250 characters
- 첫 번째 청크 미리보기:
Build a RAG agent with LangChain - Docs by LangChainSkip to main contentüöÄ Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangChain + LangGraphSearch...‚åòKAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangChainBuild a RAG agent with LangC...


### 2-2. RecursiveCharacterTextSplitter

In [7]:
# RecursiveCharacterTextSplitter 적용
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len
)

recursive_splits = recursive_text_splitter.split_documents(docs)

print(f"RecursiveCharacterTextSplitter 결과:")
print(f"- 총 청크 수: {len(recursive_splits)}")
print(f"- 첫 번째 청크 길이: {len(recursive_splits[0].page_content)} characters")
print(f"- 첫 번째 청크 미리보기:\n{recursive_splits[0].page_content[:300]}...")

RecursiveCharacterTextSplitter 결과:
- 총 청크 수: 63
- 첫 번째 청크 길이: 854 characters
- 첫 번째 청크 미리보기:
Build a RAG agent with LangChain - Docs by LangChainSkip to main contentüöÄ Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangChain + LangGraphSearch...‚åòKAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangChainBuild a RAG agent with LangC...


### 2-3. 청킹 전략 비교 분석

In [8]:
# 청킹 전략 비교
print("=" * 80)
print("청킹 전략 비교")
print("=" * 80)
print(f"\n1. CharacterTextSplitter:")
print(f"   - 총 청크 수: {len(char_splits)}")
print(f"   - 평균 청크 길이: {sum(len(chunk.page_content) for chunk in char_splits) / len(char_splits):.0f} characters")
print(f"   - 특징: 단순 구분자(\\n\\n) 기반 분할")

print(f"\n2. RecursiveCharacterTextSplitter:")
print(f"   - 총 청크 수: {len(recursive_splits)}")
print(f"   - 평균 청크 길이: {sum(len(chunk.page_content) for chunk in recursive_splits) / len(recursive_splits):.0f} characters")
print(f"   - 특징: 계층적 구분자 사용, 문맥 보존 우수")

print("\n💡 추천: RecursiveCharacterTextSplitter")
print("   - 더 일관된 청크 크기")
print("   - 문맥을 더 잘 보존")
print("   - 코드 블록과 구조를 잘 유지")

# 최종 선택
splits = recursive_splits
print(f"\n✅ 최종 선택: RecursiveCharacterTextSplitter ({len(splits)} chunks)")

청킹 전략 비교

1. CharacterTextSplitter:
   - 총 청크 수: 42
   - 평균 청크 길이: 1091 characters
   - 특징: 단순 구분자(\n\n) 기반 분할

2. RecursiveCharacterTextSplitter:
   - 총 청크 수: 63
   - 평균 청크 길이: 751 characters
   - 특징: 계층적 구분자 사용, 문맥 보존 우수

💡 추천: RecursiveCharacterTextSplitter
   - 더 일관된 청크 크기
   - 문맥을 더 잘 보존
   - 코드 블록과 구조를 잘 유지

✅ 최종 선택: RecursiveCharacterTextSplitter (63 chunks)


## 3. 임베딩 및 벡터 저장 (Milestone 3)

### 3-1. FAISS 벡터스토어

In [9]:
# OpenAI 임베딩 모델 사용
embeddings = OpenAIEmbeddings()

# FAISS 벡터스토어 생성
vectorstore_faiss = FAISS.from_documents(
    documents=splits,
    embedding=embeddings
)

print("✅ FAISS 벡터스토어 생성 완료")
print(f"   - 저장된 문서 수: {len(splits)}")
print(f"   - 임베딩 모델: OpenAI text-embedding-ada-002")

✅ FAISS 벡터스토어 생성 완료
   - 저장된 문서 수: 63
   - 임베딩 모델: OpenAI text-embedding-ada-002


### 3-2. Chroma 벡터스토어 (대안)

In [11]:
# Chroma 벡터스토어 생성 (선택사항)
vectorstore_chroma = Chroma.from_documents(
    documents=splits,
    embedding=embeddings
)

print("✅ Chroma 벡터스토어 생성 완료")
print(f"   - 저장된 문서 수: {len(splits)}")
print(f"   - 임베딩 모델: OpenAI text-embedding-ada-002")

✅ Chroma 벡터스토어 생성 완료
   - 저장된 문서 수: 63
   - 임베딩 모델: OpenAI text-embedding-ada-002


### 3-3. 벡터스토어 선택

In [12]:
# 최종 사용할 벡터스토어 선택 (FAISS 사용)
vectorstore = vectorstore_chroma
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 4}  # 상위 4개 결과 반환
)

print("✅ Retriever 설정 완료")
print(f"   - 벡터스토어: Chroma")
print(f"   - 검색 결과 수: 4")

✅ Retriever 설정 완료
   - 벡터스토어: Chroma
   - 검색 결과 수: 4


## 4. 검색 시스템 구현 및 테스트 (Milestone 4)

### 4-1. RAG 프롬프트 설정

In [13]:
# RAG 프롬프트 템플릿
prompt = PromptTemplate.from_template(
    """당신은 AI Agent와 RAG 시스템에 대한 전문가입니다. 
주어진 문맥(context)을 바탕으로 질문(question)에 정확하고 상세하게 답변해주세요.

답변 시 주의사항:
1. 주어진 문맥에서 찾을 수 있는 정보만 사용하세요.
2. 답을 모르면 "주어진 정보에서 답을 찾을 수 없습니다"라고 답하세요.
3. 기술 용어는 원문 그대로 사용하세요 (예: RAG, Vector Store, Embedding).
4. 가능하면 구체적인 예시나 코드를 포함하세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

print("✅ RAG 프롬프트 설정 완료")

✅ RAG 프롬프트 설정 완료


### 4-2. RAG Chain 구성

In [14]:
# LLM 설정
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# RAG 체인 생성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG Chain 구성 완료")
print(f"   - LLM: {llm.model_name}")
print(f"   - Temperature: {llm.temperature}")

✅ RAG Chain 구성 완료
   - LLM: gpt-4o-mini
   - Temperature: 0.0


### 4-3. 테스트 질문 (최소 3개)

In [15]:
# 테스트 질문 1: RAG 기본 개념
print("="*80)
print("질문 1: RAG란 무엇인가요?")
print("="*80)

answer1 = rag_chain.invoke("RAG란 무엇인가요? 어떻게 작동하나요?")
print(answer1)
print("\n")

질문 1: RAG란 무엇인가요?
RAG( Retrieval-Augmented Generation)란 정보 검색(data retrieval)과 생성(generation) 모델을 결합한 접근 방식입니다. RAG는 사용자의 질문에 대한 답변을 생성하기 위해 외부 데이터 소스에서 정보를 검색하고, 이 검색된 정보를 LLM(대형 언어 모델)에게 프롬프트의 일부로 전달하는 과정을 포함합니다.

RAG의 작동 방식은 다음과 같습니다:

1. **사용자 질문 입력**: 사용자가 질문을 입력합니다.
2. **정보 검색**: 입력된 질문을 바탕으로 관련 문서를 검색합니다. 이 과정에서 쿼리 변환(query transformations) 기법을 사용하여 사용자 입력을 수정하여 검색의 정확성을 높일 수 있습니다.
3. **LLM에 정보 전달**: 검색된 문서와 초기 질문을 LLM에 전달하여 답변을 생성합니다.
4. **답변 반환**: LLM이 생성한 답변을 사용자에게 반환합니다.

예를 들어, LangChain을 사용하여 RAG 에이전트를 구축할 수 있습니다. 다음은 간단한 RAG 에이전트를 구현하는 코드의 예시입니다:

```python
from langchain.tools import tool

@tool
def search_documents(query):
    # 이 함수는 주어진 쿼리에 대해 문서를 검색하는 로직을 포함합니다.
    pass

def rag_agent(user_question):
    # 문서 검색
    retrieved_docs = search_documents(user_question)
    
    # LLM에 질문과 검색된 문서 전달
    answer = llm.generate_answer(user_question, retrieved_docs)
    
    return answer
```

이와 같이 RAG는 정보 검색과 생성 모델을 결합하여 보다 정확하고 유용한 답변을 제공하는 데 도움을 줍니다.




In [16]:
# 1. 질문 입력
question = "RAG란 무엇인가요?"

# 2. Retriever가 자동으로 유사도 검색 실행
# (내부적으로)
query_embedding = OpenAIEmbeddings().embed_query(question)
# [0.234, -0.123, 0.456, ...] (1536차원)

# Chroma DB에서 유사도 검색
similar_docs = vectorstore.similarity_search_by_vector(
    embedding=query_embedding,
    k=4
)
# 결과: [doc1, doc3, doc5, doc7] (가장 유사한 4개)

# 3. 검색된 문서들을 context로 전달
context = "\n\n".join([doc.page_content for doc in similar_docs])

# 4. 프롬프트에 삽입
filled_prompt = f"""
질문: {question}

문맥: {context}

답변:
"""

# 5. LLM으로 답변 생성
answer = llm.invoke(filled_prompt)

In [17]:
print(similar_docs)
print('-'*100)
print(context)
print('-'*100)
print(answer)

[Document(id='f22f4b02-7a1b-45ca-8947-ace036e6e9d4', metadata={'source': 'https://blog.langchain.dev/deconstructing-rag/', 'language': 'en', 'title': 'Deconstructing RAG'}, page_content='information from a data source (or sources), and a process of passing the retrieved information directly to the LLM as part of the prompt (see an example prompt in LangChain hub here).ChallengeThe landscape of RAG methods has expanded greatly in recent months, resulting in some degree of overload or confusion among users about where to start and how to think about the various approaches. Over the past few months, we have worked to group RAG concepts into a few categories and have released guides for each. Below we\'ll provide a round-up of these concepts and present some future work. Major RAG themesQuery TransformationsA first question to ask when thinking about RAG: how can we make retrieval robust to variability in user input? For example, user questions may be poorly worded for the challenging task

In [18]:
# 테스트 질문 2: Text Splitter
print("="*80)
print("질문 2: Text Splitter의 종류와 차이점")
print("="*80)

answer2 = rag_chain.invoke("Text Splitter에는 어떤 종류가 있고, 각각 어떤 차이가 있나요?")
print(answer2)
print("\n")

질문 2: Text Splitter의 종류와 차이점
주어진 문맥에서 언급된 Text Splitter의 종류는 `RecursiveCharacterTextSplitter`입니다. 이 Text Splitter는 문서를 공통 구분자(예: 줄 바꿈)를 사용하여 재귀적으로 분할하여 각 청크가 적절한 크기가 되도록 합니다. 이는 일반적인 텍스트 사용 사례에 권장되는 텍스트 분할기입니다.

각 Text Splitter의 차이에 대한 구체적인 정보는 주어진 문맥에서 찾을 수 없습니다. 따라서, 주어진 정보에서 답을 찾을 수 없습니다.




In [19]:
# 테스트 질문 3: Vector Store
print("="*80)
print("질문 3: Vector Store의 역할")
print("="*80)

answer3 = rag_chain.invoke("Vector Store는 RAG 시스템에서 어떤 역할을 하나요?")
print(answer3)
print("\n")

질문 3: Vector Store의 역할
Vector Store는 RAG 시스템에서 중요한 역할을 합니다. 주어진 문맥에 따르면, Vector Store는 벡터 데이터베이스를 감싸는 래퍼로서, 임베딩을 저장하고 쿼리하는 데 사용됩니다. RAG 애플리케이션의 작동 방식은 다음과 같습니다:

1. **Retrieve (검색)**: 사용자의 입력에 따라, 관련된 데이터 조각이 Vector Store에서 검색됩니다. 이 과정에서 Retriever가 사용됩니다.
2. **Generate (생성)**: 검색된 데이터와 질문을 포함한 프롬프트를 사용하여 모델이 답변을 생성합니다.

예를 들어, 사용자가 질문을 입력하면, Vector Store는 해당 질문에 대한 관련 정보를 포함하는 블로그 포스트의 조각을 검색하여 반환할 수 있습니다. 이로 인해 RAG 시스템은 사용자 질문에 대한 적절한 답변을 제공할 수 있게 됩니다. 

또한, Vector Store는 메타데이터 필터링을 통해 구조화된 쿼리를 지원하여 비구조화된 문서를 필터링할 수 있는 기능도 제공합니다. 이러한 기능은 자연어 쿼리를 메타데이터 필드를 기반으로 한 구조화된 쿼리로 변환하는 데 유용합니다. 

결론적으로, Vector Store는 RAG 시스템에서 정보 검색과 쿼리 처리의 핵심 요소로 작용하여, 사용자에게 정확하고 관련성 높은 답변을 제공하는 데 기여합니다.




In [20]:
# 테스트 질문 4: Embedding 모델
print("="*80)
print("질문 4: Embedding 모델의 중요성")
print("="*80)

answer4 = rag_chain.invoke("Embedding 모델은 왜 중요하고, 어떻게 선택해야 하나요?")
print(answer4)
print("\n")

질문 4: Embedding 모델의 중요성
Embedding 모델은 정보 검색 및 자연어 처리에서 매우 중요한 역할을 합니다. 그 이유는 다음과 같습니다:

1. **정보 검색의 효율성**: Embedding 모델은 텍스트 데이터를 고차원 벡터로 변환하여, 유사한 의미를 가진 문서나 단어를 가까운 위치에 배치합니다. 이를 통해 검색 시 유사한 내용을 빠르게 찾을 수 있습니다.

2. **다양한 데이터 유형 지원**: 문서의 다양한 형식(예: 텍스트, 표, 이미지 등)을 처리할 수 있는 능력이 있습니다. 예를 들어, 다중 벡터 검색기를 사용하여 텍스트와 표가 혼합된 문서에서 각 표를 추출하고 요약하여 검색할 수 있습니다.

3. **모델의 성능 향상**: 적절한 chunk size를 선택함으로써 문서의 정보를 효과적으로 로드하고, LLM의 컨텍스트 윈도우에 맞게 최적화할 수 있습니다. 이는 검색 성능을 크게 향상시킬 수 있습니다.

Embedding 모델을 선택할 때 고려해야 할 사항은 다음과 같습니다:

1. **데이터의 특성**: 데이터가 어떤 형식인지, 그리고 어떤 정보를 추출하고자 하는지에 따라 적합한 모델을 선택해야 합니다. 예를 들어, 텍스트가 많은 경우 OpenAI의 `text-embedding-3-large` 모델을 사용할 수 있습니다.

2. **벡터 스토어의 선택**: Embedding 모델과 함께 사용할 벡터 스토어를 선택해야 합니다. 예를 들어, In-memory VectorStore, FAISS, Milvus 등 다양한 옵션이 있으며, 데이터의 크기와 검색 속도에 따라 적합한 스토어를 선택해야 합니다.

3. **인덱싱 전략**: 문서의 인덱싱 방법도 중요합니다. 문서를 작은 청크로 나누어 인덱싱하면 검색 효율이 높아질 수 있습니다. 예를 들어, `Text Splitters`를 사용하여 큰 문서를 작은 청크로 나누는 것이 유용합니다.

이러한 요소들을 종합적으로 고려하여 적합한 Embedding 모델과 인덱싱 전략을 선택하는 것이 중요합니

In [21]:
# 테스트 질문 5: langchain의 특성
print("="*80)
print("질문 5: langchain의 특성")
print("="*80)

answer5 = rag_chain.invoke("Langchain에서는 RAG를 어떤 방식으로 구현하나요?")
print(answer5)
print("\n")

질문 5: langchain의 특성
Langchain에서는 RAG를 다음과 같은 방식으로 구현합니다:

1. **문서 인덱싱 및 저장**: RAG 에이전트를 구축하기 위해 먼저 문서를 로드하고, 이를 적절히 분할한 후, Vector Store에 저장합니다. 이 과정은 문서의 검색 가능성을 높이는 데 필수적입니다.

2. **검색 및 생성**: RAG 에이전트는 사용자의 질문에 대해 관련 문서를 검색하고, 검색된 문서와 초기 질문을 모델에 전달하여 답변을 생성합니다. 이 과정은 다음과 같은 두 가지 방법으로 구현될 수 있습니다:
   - **단순 도구를 사용하는 RAG 에이전트**: 이 방법은 일반적인 구현으로, 사용자의 질문에 대해 정보를 검색하는 도구를 포함합니다.
   - **두 단계 RAG 체인**: 이 방법은 각 쿼리에 대해 단일 LLM 호출만을 사용하여 빠르고 효과적인 처리를 제공합니다.

3. **예시 코드**: RAG 에이전트를 구성하기 위해, 다음과 같은 간단한 도구를 구현할 수 있습니다:
   ```python
   from langchain.tools import tool

   @tool
   def search_tool(query: str):
       # Vector Store에서 쿼리에 대한 문서를 검색하는 로직
       pass
   ```

이러한 방식으로 Langchain은 RAG 에이전트를 구축하여 사용자의 질문에 대한 답변을 효과적으로 생성할 수 있습니다.




### 4-4. 스트리밍 출력 테스트

In [22]:
# 실시간 스트리밍 출력
print("="*80)
print("스트리밍 테스트: RAG 시스템 구축 시 주의사항")
print("="*80)

answer = rag_chain.stream("RAG 시스템을 구축할 때 주의해야 할 점은 무엇인가요?")
for chunk in answer:
    print(chunk, end="", flush=True)
print("\n")

스트리밍 테스트: RAG 시스템 구축 시 주의사항
RAG 시스템을 구축할 때 주의해야 할 점은 다음과 같습니다:

1. **Query Transformations**: 사용자 입력의 변동성에 강한 검색을 만들기 위해 쿼리 변환을 고려해야 합니다. 사용자 질문이 검색 작업에 적합하지 않게 잘못 표현될 수 있으므로, 쿼리 변환 기법을 사용하여 사용자 입력을 수정하여 검색 성능을 향상시킬 수 있습니다.

2. **Query Construction**: 데이터에 쿼리하기 위해 필요한 구문을 고려해야 합니다. 자연어로 된 질문은 관계형 데이터베이스나 그래프 데이터베이스와 같은 데이터 소스에서 특정 구문으로 변환되어야 합니다. 예를 들어, 자연어 질문을 SQL 요청으로 변환하는 Text-to-SQL 기술을 활용할 수 있습니다.

3. **Dynamic Query Routing**: 다양한 데이터 저장소를 통해 들어오는 쿼리를 효과적으로 라우팅하기 위해 LLM을 사용할 수 있습니다. 이는 RAG 시스템의 유연성을 높이는 데 도움이 됩니다.

4. **Document Relevance Grading**: 문서의 관련성을 평가하고 검색 쿼리를 재작성하는 단계 추가를 통해 더 깊은 제어와 사용자 맞춤화를 구현할 수 있습니다.

이러한 요소들을 고려하여 RAG 시스템을 설계하면 보다 효과적이고 사용자 친화적인 시스템을 구축할 수 있습니다.



## 5. 결과 평가 및 개선점

### 평가 기준:
1. **정확도**: 답변이 문맥에 기반하고 있는가?
2. **관련성**: 질문과 답변이 일치하는가?
3. **완전성**: 충분한 정보를 제공하는가?
4. **명확성**: 이해하기 쉬운 답변인가?

### 개선 방향:
- 더 다양한 문서 소스 추가
- 청크 크기 최적화
- 다른 임베딩 모델 실험 (HuggingFace)
- 검색 결과 수(k) 조정
- 하이브리드 검색 (키워드 + 의미 기반)

## 📊 프로젝트 완료 체크리스트

- ✅ Milestone 1: 문서 수집 및 로드 (AI Agent & RAG 블로그 5개)
- ✅ Milestone 2: 청킹 전략 비교 (CharacterTextSplitter vs RecursiveCharacterTextSplitter)
- ✅ Milestone 3: 임베딩 및 벡터 저장 (OpenAI Embeddings + FAISS)
- ✅ Milestone 4: 검색 시스템 구현 및 테스트 (4개 질문 테스트)

### 주요 성과:
- AI Agent & RAG 관련 검색 시스템 구축 완료
- 2가지 청킹 방법 비교 및 최적 방법 선정
- FAISS 벡터스토어를 활용한 효율적인 검색
- 실용적인 질문-답변 시스템 구현